In [5]:
import os
import numpy as np
from tqdm import trange
from tinygrad.nn import Conv2d, Linear, optim
from tinygrad.tensor import Tensor
from datasets import fetch_mnist
from extra.training import train, evaluate
np.random.seed(1337)
Tensor.manual_seed(1337)

In [6]:
class ConvNet:
    def __init__(self):
        self.conv1 = Conv2d(1, 10, 5)
        self.conv2 = Conv2d(10, 20, 5)
        self.fc1 = Tensor.scaled_uniform(320, 50)
        self.fc2 = Tensor.scaled_uniform(50, 10)
    
    def __call__(self, x: Tensor) -> Tensor:
        x = self.conv1(x).relu().max_pool2d(kernel_size=(2, 2))
        x = self.conv2(x).relu().max_pool2d(kernel_size=(2, 2))
        x = x.reshape(-1, 320)
        x = x.dot(self.fc1).relu()
        x = x.dot(self.fc2).logsoftmax()
        return x

In [7]:
X_train, Y_train, X_test, Y_test = fetch_mnist()
X_train = X_train.reshape(-1, 1, 28, 28) / 255.0
X_test = X_test.reshape(-1, 1, 28, 28) / 255.0

model = ConvNet()
optimizer = optim.Adam(optim.get_parameters(model), lr=0.005)

train(model, X_train, Y_train, optimizer, 100, BS=256)
evaluate(model, X_test, Y_test);

100%|██████████| 79/79 [00:07<00:00, 10.55it/s]

test set accuracy is 0.960400
